In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

from sklearn. model_selection import cross_val_score

from pprint import pprint

In [2]:
columns = ['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave_points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave_points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave_points_worst','symmetry_worst','fractal_dimension_worst']

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
df = pd.read_csv(path, names = columns)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
df.drop('id', axis = 1, inplace = True)
target = 'diagnosis'

X = df.drop(target, axis = 1)
y = df[target]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

# Check target
print('Full', y.value_counts(True), end = '\n\n')
print('Train', y_train.value_counts(True), end = '\n\n')
print('Test', y_test.value_counts(True))

Full diagnosis
B    0.627417
M    0.372583
Name: proportion, dtype: float64

Train diagnosis
B    0.626374
M    0.373626
Name: proportion, dtype: float64

Test diagnosis
B    0.631579
M    0.368421
Name: proportion, dtype: float64


In [74]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
# Really simple pipeline
# I also tried using PCA but it gave a lot worse results
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())

# Simplest way I could work out to have param grids with different solver and not get errors/warning
param_grids = [
    {
        'logisticregression__solver': ['saga'],
        'logisticregression__penalty': ['elasticnet'],
        'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
        'logisticregression__l1_ratio': [0.1, 0.4, 0.6, 0.9],  # Only for elasticnet
        'logisticregression__max_iter': [100, 500, 1000, 10000]
    },
    {
        'logisticregression__solver': ['liblinear'],
        'logisticregression__penalty': ['l1', 'l2'],  # Only l1 and l2 supported
        'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
        'logisticregression__max_iter': [100, 500, 1000, 10000]
    },
    {
        'logisticregression__solver': ['lbfgs'],
        'logisticregression__penalty': ['l2'],
        'logisticregression__C': [0.1, 0.5, 1.0, 2.0],
        'logisticregression__max_iter': [100, 500, 1000, 10000]
    }
]

gs = GridSearchCV(estimator = pipe_lr, param_grid = param_grids[2], cv = 5, n_jobs = -1, scoring = 'accuracy')
gs.fit(X_train, y_train)

print('Best from Grid Search')
print('-' * 22)
pprint(f'Best parameters: {gs.best_params_}')
print()
print(f'Best cross-validation accuracy: {(gs.best_score_ *100):.3f}%')

lr_clf = gs.best_estimator_
print()

print('Classification Report: Test Data')
print('-' * 33)
y_preds = lr_clf.predict(X_test)
print(classification_report(y_test, y_preds))

Best from Grid Search
----------------------
("Best parameters: {'logisticregression__C': 2.0, "
 "'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', "
 "'logisticregression__solver': 'lbfgs'}")

Best cross-validation accuracy: 97.582%

Classification Report: Test Data
---------------------------------
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        72
           1       0.98      0.95      0.96        42

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



---
---

In [77]:
pd.Series(y_train).value_counts()

0    285
1    170
dtype: int64